#### ¿Hacia donde se dirige el valor del `Bitcoin`?
En el ámbito financiero, la predicción del comportamiento de los activos digitales como el Bitcoin es un desafío complejo debido a la alta volatilidad en su mercado, sumado a que(cabe aclarar) no es un campo de estudio en el que estoy capacitado formalmente. De todas formas, las series temporales son algo inherente a este terreno y su análisis exploratorio permite poder generar especulaciones con fundamento. 

La predicción del futuro a partir de indicios es uno de los principales objetivos en las series de tiempo, métodos utilizados como el ingenuo, de la media, de la deriva y de la media móvil simple se consideran muy básicos y que no aportan variabilidad en los resultados. Sin embargo, el Aprendizaje Automático también se emplea como solución a la hora de querer generar pronósticos efectivos y al igual que en los modelos convencionales se requiere como base comprender la infromación en cuestión para tomar el caminos correcto.

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input


df = pd.read_csv("data/bitcoin_usd.csv")

df["date"] = pd.to_datetime(df["date"], format='%Y-%m-%d')
df.sort_values(by="date", inplace=True)

df 

#### Análisis Exploratorio de series temporales

- Nivel: media estadística de la serie.

- Estacionariedad: componente en series cuyas medidas(nivel y desviación éstandar) no cambian con el tiempo.

- Tendencia: dirección general y el cambio a largo plazo del nivel de la serie independientemente de las fluctuaciones en esta misma.

- Estacionalidad: picos y valles que se repiten periódicamente(diarios, semanales, anuales, etc), es decir, siguiendo un patrón.

- Ruido blanco: al igual que un conjunto de datos convencional, sufren de errores aleatorios e imprecisiones que no tienen un origen concreto.

In [ ]:
df.set_index("date", inplace=True)

fig, ax = plt.subplots(5,1, figsize=(9,18))

def trend(serie, index):
    ax[index].plot(df[serie], label="Serie temporal")
    ax[index].plot(seasonal_decompose(df[serie], model="aditive").trend, label="Tendencia")
    ax[index].set_ylabel(serie)
    ax[index].set_xlabel("dates")
    ax[index].legend()
    ax[index].grid("on")

trend("volume", 0)
trend("open_USD", 1)
trend("close_USD", 2)
trend("high_USD", 3)
trend("low_USD", 4)

plt.subplots_adjust(hspace=0.4)
plt.show()

#### Tendencia evidente

a excepción de la variable 'volume', todas las demás series presentan una tendencia que va en aumento, sobre todo de forma notable en el último período. A su vez, podemos anular una posible estacionariedad debido al constante cambio en la media y desvación éstandar, un método utilizado es la prueba de Dickey-Fuller en estas suposiciones pero en este caso visualmente es muy notoria la no unitariedad.

______________________________________________________________________________________________________________________________

#### Análisis de estacionalidad 

Autocorrelación: medida que se basa en la correlación lineal entre los valores de una serie temporal en diferentes períodos, es decir, una versión desfasada o con retraso y otra más reciente.

Función de autocorrelación: correlograma que visualiza un conjunto de autocorrelaciones con sus respectivos retardos, los incrementos y decrementos conforme avanzan los retardos ayudan a identifincar un patrón estacional en una serie de tiempo

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(20,65))

def correlogram(serie, index):
    plot_acf(df[serie], lags=df.shape[0] - 400, ax=ax[index])
    ax[index].set_title(serie)
    ax[index].set_ylabel("Autocorrelación")
    ax[index].set_xlabel("Retardo(K)")

correlogram("volume", 0)
correlogram("open_USD", 1)
correlogram("close_USD", 2)
correlogram("high_USD", 3)
correlogram("low_USD", 4)

plt.subplots_adjust(hspace=0.3)
plt.show()

#### Estacionalidad vs Ciclicidad 
Al observar los diferentes corregloramas no podemos asimilar una estacionalidad clara, a su vez los valores negativos denotan cierto ruido o cambios bruscos en algunas partes del progreso. En su lugar, las respectivas series no deberían considerarse estacionales sino más bien cíclicas, estas a diferencia de las series temporales con estacionalidad éstandar contienen patrones que se repiten en intervalos de tiempo irregulares o variables, pueden ser causados por factores internos y su duración es más prolongada.

______________________________________________________________________________________________________________________________

#### Dashboard que traza un análisis de tendencia mensual junto con un pronóstico de OHLC y volumen de trading

In [ ]:
df.reset_index(inplace=True)

max_month_mean = html.B(children=[], id="max")
min_month_mean = html.B(children=[], id="min")

options_dropdown = [
    {"label":"Volúmenes de trading","value":"volume"},
    {"label":"Valores de apertura","value":"open_USD"},
    {"label":"Valores de cierre","value":"close_USD"},
    {"label":"Valores más bajos","value":"low_USD"},
    {"label":"Valores más altos","value":"high_USD"}
]

app = dash.Dash(__name__)
app.layout =  html.Div(id="body", className="e7_body", children=[
    html.H1("Análisis de Tendencia Mensual", id="H1", className="e7_title"),
    html.Div(id="KPI_div", className="e7_KPI_div", children=[
        html.P(max_month_mean, className="e7_KPI"),
        html.P(min_month_mean, className="e7_KPI")
    ]),
    html.Div(className="e7_graph_div", children=[
       dcc.Dropdown(id="dropdown_1", className="e7_dropdown_1",
                        options=[
                            {"label":"Vólumen de trading (promedio)","value":"volume"},
                            {"label":"Valor de cierre (promedio)","value":"close_USD"}
                        ],
                        value="volume",
                        multi=False,
                        clearable=False),
        dcc.Graph(id="trend_analysis", figure={}, className="e7_graph_1")
    ]),
    html.H2([html.Span("Pronóstico", className="e7_title", style={"color":"yellow"})," del próximo trimestre ($)"], id="H2", className="e7_title"),
    html.Div(id="e7_dropdown_div", className="e7_dropdown_div", children=[
       dcc.Dropdown(id="dropdown_2", className="e7_dropdown_2",
                        options= options_dropdown,
                        value="volume",
                        multi=False,
                        clearable=False) 
    ]),
    dcc.Graph(id="forecasting", figure={}, className="e7_graph_2")
])

@app.callback(
    [Output(component_id="max",component_property="children"),
    Output(component_id="min",component_property="children"),
    Output(component_id="trend_analysis",component_property="figure"),
    Output(component_id="forecasting",component_property="figure")],
    [Input(component_id="dropdown_1",component_property="value"),
    Input(component_id="dropdown_2",component_property="value")]
)

def update_graph(slct_var, slct_var_forecast):
    
    df_seg = df.groupby(pd.Grouper(key="date", freq="M"))[slct_var].mean().reset_index()
    df_seg["month"] = df_seg["date"].dt.strftime("%b")
    df_seg["month_year"] = df_seg["date"].dt.strftime("%b %Y")

    month_mean = df_seg.groupby("month")[slct_var].mean().reset_index()

    max_mean = str(round(month_mean.max()[1], 1))
    max_month = month_mean.max()[0]
    max_month_mean = f"Máx.: {max_month}({max_mean})"

    min_mean = str(round(month_mean.min()[1], 1))
    min_month = month_mean.min()[0]
    min_month_mean = f"Mín.: {min_month}({min_mean})"
    
    months_means = go.Figure()
    months_means.add_trace(go.Scatter(x=df_seg["month_year"], y=df_seg[slct_var], mode="lines", fill="tozeroy", fillcolor="rgba(0, 255, 0, 0.2)"))
    months_means.update_layout(title_text="Promedios de meses",  xaxis_title=" ", yaxis_title=" ")
    
    if slct_var == "volume":
        TES = ExponentialSmoothing(df[slct_var_forecast], seasonal="add", seasonal_periods=255)
    elif slct_var == "high_USD":
        TES = ExponentialSmoothing(df[slct_var_forecast], trend="add", seasonal="add", seasonal_periods=335)
    else:
        TES = ExponentialSmoothing(df[slct_var_forecast], trend="add", seasonal="add", seasonal_periods=325)
    
    forecasting_model = TES.fit()
    
    new_dates = list(pd.date_range("2021-01-31","2021-05-01"))
    forecast = forecasting_model.forecast(90)
    
    selected_label = next(
        (option["label"] for option in options_dropdown if option["value"] == slct_var_forecast),
        "No encontrado" 
     )
    
    forecasting = go.Figure()
    forecasting.add_trace(go.Scatter(x=df["date"], y=df[slct_var_forecast], mode="lines", fill="tozeroy", fillcolor="rgba(0, 0, 255, 0.2)", name="Valores históricos"))
    forecasting.add_trace(go.Scatter(x=new_dates, y=forecast, mode="lines", fill="tozeroy", fillcolor="rgba(255, 0, 0, 0.2)", name="Pronóstico de tres meses"))
    forecasting.update_layout(title_text=f"Pronóstico de {selected_label}",  xaxis_title=" ", yaxis_title=" ")

    return max_month_mean, min_month_mean, months_means, forecasting

if __name__ == "__main__":
    app.run_server(debug=False)

#### Suavización Exponencial

La técnica implementada basa su método en precisamente suavizar las fluctuaciones de la serie a través de una combinación ponderada de valores anteriores, los parámetros que utiliza son el factor de suavización(Valor Alpha) y el Nivel inicial(0). Dependiendo del comportamiento de la serie el modelo generado a partir de esta técnica puede ir variando y añadiendo más componentes, algunas variaciones son: Suavizado Simple, Tendencia lineal de Holt, Tendencia amortiguada, Estacional Simple o Aditivo de Winters.

Suavizado Exponencial Triple o Estacional Simple: modelo apropiado para series de tiempo con tendencia y un efecto estacional que es constante a lo largo del tiempo, sus hiperparámetros correspondientes son el nivel, la tendencia y la estacionalidad, además se debe especificar el intervalo de tiempo en que transcurre el patrón. En este caso, es importante tener en cuenta la complejidad y volatilidad en el valor del Bitcoin, cuyas series analizadas han experimentado constantes variaciones y períodos cíclicos, sumado a que no se dispone de la parte escencialmente técnica del mercado cripto, lo que provoca una ajuste de hiperparámetros más complejo en el modelo debido a la dificultad de detectar y capturar la estructura subyacente de la serie.